In [1]:
%%capture
%pip install "sentence-transformers>=3.0.0" haystack-ai google-ai-haystack wikipedia rich google-generativeai

In [47]:
%%capture

import os
import pandas as pd
import random
import time
import wikipedia

from rich import print

from datasets import load_dataset
from dotenv import load_dotenv
from google.generativeai import GenerationConfig
from haystack import Document, Pipeline
from haystack.components.builders import PromptBuilder
from haystack.components.embedders import SentenceTransformersDocumentEmbedder, SentenceTransformersTextEmbedder
from haystack.components.generators import AzureOpenAIGenerator
from haystack.components.retrievers import InMemoryEmbeddingRetriever
from haystack.components.retrievers.in_memory import InMemoryBM25Retriever
from haystack.components.preprocessors import DocumentCleaner, DocumentSplitter
from haystack.components.writers import DocumentWriter
from haystack.document_stores.in_memory import InMemoryDocumentStore
from haystack.document_stores.types import DuplicatePolicy
from haystack_integrations.components.generators.google_ai import GoogleAIGeminiGenerator, GoogleAIGeminiChatGenerator


## Authorization

- You need an Google API Key, one can be obtained at https://aistudio.google.com/app/apikey
- [Optional] You also need an Azure OpenAI API Key and Azure OpenAI endpoint for the second part of this notebook.

In [3]:
load_dotenv()

True

## Index Documents

### Load data from Wikipedia

In [4]:
favourite_bands="""Audioslave
Blink-182
Dire Straits
Evanescence
Green Day
Muse (band)
Nirvana (band)
Sum 41
The Cure
The Smiths""".split("\n")

In [5]:
raw_docs=[]

for title in favourite_bands:
    page = wikipedia.page(title=title, auto_suggest=False)
    doc = Document(content=page.content, meta={"title": page.title, "url":page.url})
    raw_docs.append(doc)

### Indexing Pipeline

In [6]:
document_store = InMemoryDocumentStore()

In [7]:
embedder = SentenceTransformersDocumentEmbedder("sentence-transformers/all-MiniLM-L6-v2", progress_bar=False)
embedder.warm_up()

In [8]:
indexing = Pipeline()
indexing.add_component("cleaner", DocumentCleaner())
indexing.add_component("splitter", DocumentSplitter(split_by='sentence', split_length=2))
indexing.add_component("embedder", embedder)
indexing.add_component("writer", DocumentWriter(document_store=document_store, policy=DuplicatePolicy.OVERWRITE))
indexing.connect("cleaner", "splitter")
indexing.connect("splitter", "embedder")
indexing.connect("embedder", "writer")

🚅 Components
  - cleaner: DocumentCleaner
  - splitter: DocumentSplitter
  - embedder: SentenceTransformersDocumentEmbedder
  - writer: DocumentWriter
🛤️ Connections
  - cleaner.documents -> splitter.documents (List[Document])
  - splitter.documents -> embedder.documents (List[Document])
  - embedder.documents -> writer.documents (List[Document])

In [9]:
indexing.run({"cleaner":{"documents":raw_docs}})

c:\sw\anaconda3\envs\torch\Lib\site-packages\transformers\models\bert\modeling_bert.py:440: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:555.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


{'writer': {'documents_written': 1610}}

In [10]:
print(document_store.filter_documents()[0])

Document(id=c21fdea0ecc6fabdfff42c479ed364ae892713273369bf044636b7fea643d8bd, content: 'Audioslave was an American 
rock supergroup formed in Glendale, California, in 2001. The four-piece b...', meta: {'title': 'Audioslave', 'url':
'https://en.wikipedia.org/wiki/Audioslave', 'source_id': 
'cf53c7ec310b6c605f6528b4edb9698b78896db7725e19e65c86ee6a871d5e10', 'page_number': 1, 'split_id': 0, 
'split_idx_start': 0}, embedding: vector of size 384)

# Experiment Prompts

In [11]:
prompt_template = """
Using the information contained in the context, give a comprehensive answer to the question.
If the answer is contained in the context, also report the source URL.
If the answer cannot be deduced from the context, do not give an answer.

Context:
  {% for doc in documents %}
  {{ doc.content }} URL:{{ doc.meta['url'] }}
  {% endfor %};
  Question: {{query}}
"""

critic_prompt_template = """
Decide if the following answer is consistent with the corresponding sources. Note that 
consistency means all information in the answer is supported by the sources.

Sources: [
  {% for doc in documents %}
  {{ doc.content }} URL:{{ doc.meta['url'] }}
  {% endfor %};
]
Answer: [{{answer}}]

Explain your reasoning step by step then answer [yes/no] the question.
ANSWER:
"""

questions="""What was the original name of Sum 41?
What is the name of the lead singer of Blink-182?
What was the title of Nirvana's breakthrough album released in 1991?
What does Rhodes Statue look like?
Green Day's "American Idiot" is a rock opera. What's the story it tells?
Who was the lead singer of The Cure?
Audioslave was formed by members of two iconic bands. Can you name the bands and discuss the sound of Audioslave in comparison?
Evanescence's "Bring Me to Life" features a male vocalist. Who is he, and how does his voice complement Amy Lee's in the song?
What is Sum 41's debut studio album called?
How many languages are there?
What did Einstein win the Nobel Prize for?
What What does Rhodes Statue look like?
Who was the lead singer of Audioslave?
Who has the most followers on Instagram?
When was Nirvana's first studio album, "Bleach," released?
Were the Smiths an influential band?
What is the name of Evanescence's debut album?
Which band was Morrissey the lead singer of before he formed The Smiths?
Whose the father of Luke Skywalker?
What type of music plays Coldplay?
Dire Straits' hit song "Money for Nothing" features a guest vocal by a famous artist. Who is this artist?
Who played the song "Like a stone"?
What is the most interesting thing you know?""".split('\n')

In [12]:
def get_generative_answer(query, rag_model):
  results = rag_model.run({
      "text_embedder": {"text": query},
      "prompt_builder": {"query": query}
    }
  )

  answer = results["llm"]["replies"][0]  
  return answer

def get_critic_answer(query, rag_model, critic_model):  
  model_answer = get_generative_answer(query, rag_model)
  results = critic_model.run({
      "text_embedder": {"text": query},
      "prompt_builder": {"answer": model_answer}
    }
  )
  critic_answer = results["llm"]["replies"][0]  
  return model_answer, critic_answer


In [22]:
experiment_results = {
    "model_name": [],
    "question": [],
    "model_answer": [],
    "critic_answer": []
}

# RAG with Gemini 1.5-flash

In [16]:
GEMINI_FLASH = "gemini-1.5-flash-latest"

generation_config = GenerationConfig(temperature=0.0, top_p=0.9, candidate_count=1)

def get_gemini_model(model=GEMINI_FLASH, generation_config=generation_config):
    return GoogleAIGeminiGenerator(model=GEMINI_FLASH, generation_config=generation_config)    


print(f"Gemini model {get_gemini_model()._model_name} loaded.")

Gemini model gemini-1.5-flash-latest loaded.

In [27]:
prompt_builder = PromptBuilder(template=prompt_template)
gemini_rag = Pipeline()
gemini_rag.add_component("text_embedder", SentenceTransformersTextEmbedder("sentence-transformers/all-MiniLM-L6-v2", progress_bar=False))
gemini_rag.add_component("retriever", InMemoryEmbeddingRetriever(document_store=document_store, top_k=5))
gemini_rag.add_component("prompt_builder", prompt_builder)
gemini_rag.add_component("llm", get_gemini_model())

gemini_rag.connect("text_embedder.embedding", "retriever.query_embedding")
gemini_rag.connect("retriever", "prompt_builder.documents")
gemini_rag.connect("prompt_builder", "llm")


🚅 Components
  - text_embedder: SentenceTransformersTextEmbedder
  - retriever: InMemoryEmbeddingRetriever
  - prompt_builder: PromptBuilder
  - llm: GoogleAIGeminiGenerator
🛤️ Connections
  - text_embedder.embedding -> retriever.query_embedding (List[float])
  - retriever.documents -> prompt_builder.documents (List[Document])
  - prompt_builder.prompt -> llm.parts (str)

### Critict model

In [18]:
gemini_critic = Pipeline()
gemini_critic.add_component("text_embedder", SentenceTransformersTextEmbedder("sentence-transformers/all-MiniLM-L6-v2", progress_bar=False))
gemini_critic.add_component("retriever", InMemoryEmbeddingRetriever(document_store=document_store, top_k=5))
gemini_critic.add_component("prompt_builder", PromptBuilder(template=critic_prompt_template))
gemini_critic.add_component("llm", get_gemini_model())

gemini_critic.connect("text_embedder.embedding", "retriever.query_embedding")
gemini_critic.connect("retriever", "prompt_builder.documents")
gemini_critic.connect("prompt_builder", "llm")

🚅 Components
  - text_embedder: SentenceTransformersTextEmbedder
  - retriever: InMemoryEmbeddingRetriever
  - prompt_builder: PromptBuilder
  - llm: GoogleAIGeminiGenerator
🛤️ Connections
  - text_embedder.embedding -> retriever.query_embedding (List[float])
  - retriever.documents -> prompt_builder.documents (List[Document])
  - prompt_builder.prompt -> llm.parts (str)

### Expermient with gemini-1.5-flash-latest

In [ ]:
%%time
MAX_RPM = 14
request_count = 0

for i, q in enumerate(questions):  
  print(f"Question {i+1}/{len(questions)}: {q}")
  model_answer, critic_answer = get_critic_answer(q, gemini_rag, gemini_critic)

  experiment_results["model_name"].append(GEMINI_FLASH)
  experiment_results["question"].append(q)
  experiment_results["model_answer"].append(model_answer)
  experiment_results["critic_answer"].append(critic_answer)

  request_count += 2 # 2 requests per question, one for the answer and one for the critic
  if request_count >= MAX_RPM:    
    request_count = 0
    time.sleep(60) 

Question 1/23: What was the original name of Sum 41?

Question 2/23: What is the name of the lead singer of Blink-182?

Question 3/23: What was the title of Nirvana's breakthrough album released in 1991?

Question 4/23: What does Rhodes Statue look like?

Question 5/23: Green Day's "American Idiot" is a rock opera. What's the story it tells?

...

Question 20/23: What type of music plays Coldplay?

Question 21/23: Dire Straits' hit song "Money for Nothing" features a guest vocal by a famous artist. Who is this 
artist?

Question 22/23: Who played the song "Like a stone"?

Question 23/23: What is the most interesting thing you know?

CPU times: total: 2.03 s
Wall time: 3min 32s


In [ ]:
gemini_experiment_results_df = pd.DataFrame(experiment_results)
gemini_experiment_results_df

,model_name,question,model_answer,critic_answer
0,gemini-1.5-flash-latest,What was the original name of Sum 41?,The context does not provide the original name...,The answer claims that the context does not pr...
1,gemini-1.5-flash-latest,What is the name of the lead singer of Blink-182?,The lead singer of Blink-182 is **Tom DeLonge*...,The answer states that Tom DeLonge is the lead...
2,gemini-1.5-flash-latest,What was the title of Nirvana's breakthrough a...,Nirvana's breakthrough album released in 1991 ...,Reasoning:\n\n1. The answer states that Nirvan...
3,gemini-1.5-flash-latest,What does Rhodes Statue look like?,The context does not provide information about...,The answer states that the context does not pr...
4,gemini-1.5-flash-latest,"Green Day's ""American Idiot"" is a rock opera. ...","The rock opera ""American Idiot"" follows the jo...","Reasoning:\n\n1. The answer states that ""Ameri..."
5,gemini-1.5-flash-latest,Who was the lead singer of The Cure?,Robert Smith is the lead singer of The Cure. \...,The answer states that Robert Smith is the lea...
6,gemini-1.5-flash-latest,Audioslave was formed by members of two iconic...,Audioslave was formed by members of **Soundgar...,Here's a breakdown of the answer's consistency...
7,gemini-1.5-flash-latest,"Evanescence's ""Bring Me to Life"" features a ma...",The context states that Evanescence's label in...,The answer states that Amy Lee reluctantly agr...
8,gemini-1.5-flash-latest,What is Sum 41's debut studio album called?,Sum 41's debut studio album is called **All Ki...,The answer states that Sum 41's debut studio a...
9,gemini-1.5-flash-latest,How many languages are there?,This question cannot be answered from the give...,"The answer ""This question cannot be answered f..."


____________________________
# Rag with GPT-4o

In [48]:
import os

OPENAI_DEPLOYMENT = 'gpt-4o'
azure_openai_generation_kwargs = {
    'temperature': 0.0,
    'top_p': 0.9,
    'n': 1
}
def get_azure_model(generation_kwargs=azure_openai_generation_kwargs):
    return AzureOpenAIGenerator(azure_deployment=OPENAI_DEPLOYMENT, generation_kwargs=generation_kwargs)

### RAG Model

In [30]:
gpt4o_rag = Pipeline()
gpt4o_rag.add_component("text_embedder", SentenceTransformersTextEmbedder("sentence-transformers/all-MiniLM-L6-v2", progress_bar=False))
gpt4o_rag.add_component("retriever", InMemoryEmbeddingRetriever(document_store=document_store, top_k=5))
gpt4o_rag.add_component("prompt_builder", PromptBuilder(template=prompt_template))
gpt4o_rag.add_component("llm", get_azure_model())

gpt4o_rag.connect("text_embedder.embedding", "retriever.query_embedding")
gpt4o_rag.connect("retriever", "prompt_builder.documents")
gpt4o_rag.connect("prompt_builder", "llm")

🚅 Components
  - text_embedder: SentenceTransformersTextEmbedder
  - retriever: InMemoryEmbeddingRetriever
  - prompt_builder: PromptBuilder
  - llm: AzureOpenAIGenerator
🛤️ Connections
  - text_embedder.embedding -> retriever.query_embedding (List[float])
  - retriever.documents -> prompt_builder.documents (List[Document])
  - prompt_builder.prompt -> llm.prompt (str)

### Critic Model

In [31]:
gpt4o_critic = Pipeline()
gpt4o_critic.add_component("text_embedder", SentenceTransformersTextEmbedder("sentence-transformers/all-MiniLM-L6-v2", progress_bar=False))
gpt4o_critic.add_component("retriever", InMemoryEmbeddingRetriever(document_store=document_store, top_k=5))
gpt4o_critic.add_component("prompt_builder", PromptBuilder(template=critic_prompt_template))
gpt4o_critic.add_component("llm", get_azure_model())

gpt4o_critic.connect("text_embedder.embedding", "retriever.query_embedding")
gpt4o_critic.connect("retriever", "prompt_builder.documents")
gpt4o_critic.connect("prompt_builder", "llm")

🚅 Components
  - text_embedder: SentenceTransformersTextEmbedder
  - retriever: InMemoryEmbeddingRetriever
  - prompt_builder: PromptBuilder
  - llm: AzureOpenAIGenerator
🛤️ Connections
  - text_embedder.embedding -> retriever.query_embedding (List[float])
  - retriever.documents -> prompt_builder.documents (List[Document])
  - prompt_builder.prompt -> llm.prompt (str)

__________________________________________________________________

### Expermient with gpt-4O

In [33]:
%%time

for i, q in enumerate(questions):  
  print(f"Question {i+1}/{len(questions)}: {q}")  
  model_answer, critic_answer = get_critic_answer(q, gpt4o_rag, gpt4o_critic)  
  experiment_results["model_name"].append(OPENAI_DEPLOYMENT)
  experiment_results["question"].append(q)
  experiment_results["model_answer"].append(model_answer)
  experiment_results["critic_answer"].append(critic_answer)


Question 1/23: What was the original name of Sum 41?

Question 2/23: What is the name of the lead singer of Blink-182?

Question 3/23: What was the title of Nirvana's breakthrough album released in 1991?

Question 4/23: What does Rhodes Statue look like?

Question 5/23: Green Day's "American Idiot" is a rock opera. What's the story it tells?

...

Question 20/23: What type of music plays Coldplay?

Question 21/23: Dire Straits' hit song "Money for Nothing" features a guest vocal by a famous artist. Who is this 
artist?

Question 22/23: Who played the song "Like a stone"?

Question 23/23: What is the most interesting thing you know?

CPU times: total: 2.3 s
Wall time: 2min 4s


# Experiment Results

In [ ]:
experiment_results_df = pd.DataFrame(experiment_results)
experiment_results_df.to_csv("experiment_results.csv")
experiment_results_df

,model_name,question,model_answer,critic_answer
0,gemini-1.5-flash-latest,What was the original name of Sum 41?,The context does not provide the original name...,The answer claims that the context does not pr...
1,gemini-1.5-flash-latest,What is the name of the lead singer of Blink-182?,The lead singer of Blink-182 is **Tom DeLonge*...,The answer states that Tom DeLonge is the lead...
2,gemini-1.5-flash-latest,What was the title of Nirvana's breakthrough a...,Nirvana's breakthrough album released in 1991 ...,Reasoning:\n\n1. The answer states that Nirvan...
3,gemini-1.5-flash-latest,What does Rhodes Statue look like?,The context does not provide information about...,The answer states that the context does not pr...
4,gemini-1.5-flash-latest,"Green Day's ""American Idiot"" is a rock opera. ...","The rock opera ""American Idiot"" follows the jo...","Reasoning:\n\n1. The answer states that ""Ameri..."
5,gemini-1.5-flash-latest,Who was the lead singer of The Cure?,Robert Smith is the lead singer of The Cure. \...,The answer states that Robert Smith is the lea...
6,gemini-1.5-flash-latest,Audioslave was formed by members of two iconic...,Audioslave was formed by members of **Soundgar...,Here's a breakdown of the answer's consistency...
7,gemini-1.5-flash-latest,"Evanescence's ""Bring Me to Life"" features a ma...",The context states that Evanescence's label in...,The answer states that Amy Lee reluctantly agr...
8,gemini-1.5-flash-latest,What is Sum 41's debut studio album called?,Sum 41's debut studio album is called **All Ki...,The answer states that Sum 41's debut studio a...
9,gemini-1.5-flash-latest,How many languages are there?,This question cannot be answered from the give...,"The answer ""This question cannot be answered f..."


## Context sentences per question

In [46]:
%%time

context_per_question = {    
    "question": [],
    "doc1": [],
    "doc2": [],
    "doc3": [],
    "doc4": [],
    "doc5": []
}

query_pipeline = Pipeline()
query_pipeline.add_component("text_embedder", SentenceTransformersTextEmbedder("sentence-transformers/all-MiniLM-L6-v2", progress_bar=False))
query_pipeline.add_component("retriever", InMemoryEmbeddingRetriever(document_store=document_store, top_k=5))

query_pipeline.connect("text_embedder.embedding", "retriever.query_embedding")

def get_context_for_question(question, query_pipeline=query_pipeline):
  results = query_pipeline.run({"text_embedder": {"text": question}})
  return [(doc.content, doc.meta['url']) for doc in results["retriever"]["documents"]]

for i, q in enumerate(questions):
  # print(f"Question {i+1}/{len(questions)}")
  context_per_question["question"].append(q)
  for n, doc in enumerate(get_context_for_question(q)):
    content, url = doc
    context_per_question[f"doc{n+1}"].append(f'{content} URL: {url}')
    # print(f'"{content.replace("\n", "").lstrip()}"\nURL: {url}')

context_per_question_df = pd.DataFrame(context_per_question)
context_per_question_df.to_csv("context_per_question.csv")
context_per_question_df

CPU times: total: 547 ms
Wall time: 1.64 s


,question,doc1,doc2,doc3,doc4,doc5
0,What was the original name of Sum 41?,"Sum 41 is a Canadian rock band from Ajax, Onta...","\nIn 1999, Sum 41 signed an international reco...","\nIn honor of Pelletier, Sum 41 named its next...",Sum 41 was nominated for a Grammy Award for B...,", Rob Base and DJ E-Z Rock, Metallica, Guns N'..."
1,What is the name of the lead singer of Blink-182?,Blink-182 is an American rock band formed in P...,== History == === Formation and initial years...,"] their presence is everywhere."" ""When it come...",After legal battles with DeLonge were worked ...,\nblink-182's straightforward approach and sim...
2,What was the title of Nirvana's breakthrough a...,"\nCharacterized by a punk aesthetic, Nirvana's...","\nIn the late 1980s, Nirvana established itsel...",The album topped both the US and UK album cha...,""" === 1991–1992: Nevermind and mainstream brea...",The songs are now about conflicts in relation...
3,What does Rhodes Statue look like?,"He described them as ""inferior sketches of wo...","For their second album, Origin of Symmetry (2...","Eight days later, they put out a teaser video...",To celebrate the tenth anniversary of their a...,They undertook a second North American leg al...
4,"Green Day's ""American Idiot"" is a rock opera. ...","Green Day's seventh album, a rock opera calle...",The show features an expanded story of the or...,Backed by the success of the album's first si...,""" He concluded that ""American Idiot, despite i...","\nIn 2009, the band collaborated with theater ..."
5,Who was the lead singer of The Cure?,The Cure are an English rock band formed in Cr...,(1989). The Cure: Songwords 1978–1989. URL: h...,"Melody Maker praised the album as ""psychedeli...",Artists who have cited their influence by or ...,"Released in late 1984, the Cure's first live ..."
6,Audioslave was formed by members of two iconic...,""" == Musical style and influences ==\nAudiosla...",Critics first described Audioslave as a combi...,They were characterised by his cryptic approa...,Audioslave was an American rock supergroup for...,The new funk and soul influences were also we...
7,"Evanescence's ""Bring Me to Life"" features a ma...",Amy is very artistic and never has had a prob...,AllMusic's Stephen Thomas Erlewine wrote of E...,The band recorded the album at Blackbird Stud...,"\nA few weeks later, the label relented, infor...",Evanescence is an American rock band founded i...
8,What is Sum 41's debut studio album called?,"Sum 41 is a Canadian rock band from Ajax, Onta...","Following the album's release, the band went ...","\nIn 1999, Sum 41 signed an international reco...","\nIn honor of Pelletier, Sum 41 named its next...",Sum 41 was nominated for a Grammy Award for B...
9,How many languages are there?,4 in the US and No. 8 in the United Kingdom. ...,. possibly five shows. URL: https://en.wikiped...,"There's not an accent then on the music, ther...","It has sold more than 17 million worldwide, i...","7 US), and ""Your Latest Trick"" (No. 26 UK). U..."


___________________________________________